# 特征工程——Rental Listing Inquiries 数据集
其中房屋的特征 x 共有 14 维，响应值 y 为用户对该公寓的感兴趣程度（高、中、低三类）。

去掉了一些无用特征，对日期转换后，发现 year 相同，对分类没有作用，所以也去掉。
对数字类型特征进行了标准化。

In [16]:
# 首先导入必要的包包（土豆对工具包的昵称，哈哈：）
import numpy as np
import pandas as pd

#用于计算feature字段的文本特征提取
from sklearn.feature_extraction.text import  CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer

#CountVectorizer为稀疏特征，特征编码结果存为稀疏矩阵 xgboost 处理更高效
from scipy import sparse

#对类别型特征进行编码
from sklearn.preprocessing import LabelEncoder
#from MeanEncoder import MeanEncoder

#对地理位置通过聚类进行离散化
from sklearn.cluster import KMeans
from nltk.metrics import distance as distance

# 数据标准化
from sklearn.preprocessing import StandardScaler

## 读取数据 & 数据探索

In [17]:
dpath = './data/'
# 读取训练数据
train = pd.read_json(dpath +"RentListingInquries_train.json")
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [18]:
# 读取测试数据
test = pd.read_json(dpath +"RentListingInquries_test.json")
test.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:41,Large with awesome terrace--accessible via bed...,Suffolk Street,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,[https://photos.renthop.com/2/7142618_1c45a2c8...,2950,99 Suffolk Street
1,1.0,2,0,2016-06-24 06:36:34,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,[https://photos.renthop.com/2/7210040_d824cc71...,2850,176 Thompson Street
100,1.0,1,3dbbb69fd52e0d25131aa1cd459c87eb,2016-06-03 04:29:40,New York chic has reached a new level ...,101 East 10th Street,"[Doorman, Elevator, No Fee]",40.7306,7103890,-73.9890,9ca6f3baa475c37a3b3521a394d65467,[https://photos.renthop.com/2/7103890_85b33077...,3758,101 East 10th Street
1000,1.0,2,783d21d013a7e655bddc4ed0d461cc5e,2016-06-11 06:17:35,Step into this fantastic new Construction in t...,South Third Street\r,"[Roof Deck, Balcony, Elevator, Laundry in Buil...",40.7109,7143442,-73.9571,0b9d5db96db8472d7aeb67c67338c4d2,[https://photos.renthop.com/2/7143442_0879e9e0...,3300,251 South Third Street\r
100000,2.0,2,6134e7c4dd1a98d9aee36623c9872b49,2016-04-12 05:24:17,"~Take a stroll in Central Park, enjoy the ente...","Midtown West, 8th Ave","[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.7650,6860601,-73.9845,b5eda0eb31b042ce2124fd9e9fcfce2f,[https://photos.renthop.com/2/6860601_c96164d8...,4900,260 West 54th Street


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 10 to 99994
Data columns (total 15 columns):
bathrooms          49352 non-null float64
bedrooms           49352 non-null int64
building_id        49352 non-null object
created            49352 non-null object
description        49352 non-null object
display_address    49352 non-null object
features           49352 non-null object
interest_level     49352 non-null object
latitude           49352 non-null float64
listing_id         49352 non-null int64
longitude          49352 non-null float64
manager_id         49352 non-null object
photos             49352 non-null object
price              49352 non-null int64
street_address     49352 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 6.0+ MB


In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74659 entries, 0 to 99999
Data columns (total 14 columns):
bathrooms          74659 non-null float64
bedrooms           74659 non-null int64
building_id        74659 non-null object
created            74659 non-null object
description        74659 non-null object
display_address    74659 non-null object
features           74659 non-null object
latitude           74659 non-null float64
listing_id         74659 non-null int64
longitude          74659 non-null float64
manager_id         74659 non-null object
photos             74659 non-null object
price              74659 non-null int64
street_address     74659 non-null object
dtypes: float64(3), int64(3), object(8)
memory usage: 8.5+ MB


训练集和测试集均没有缺失值.

测试集没有 interest_level（类型特征，要预测的目标），其他和训练数据一样。

共有 14 个特征，数据类型 6 个数字型，8 个字符串型。

### 将类别型的标签interest_level编码为数字

In [21]:
y_map = {'low': 2, 'medium': 1, 'high': 0}
# 采用匿名函数 lambda
train['interest_level'] = train['interest_level'].apply(lambda x: y_map[x])

In [22]:
# 用于训练模型
y_train = train['interest_level']
#保留，生成提交文件时需要 
test_id = test['listing_id']

### 删除无用特征

根据特征的含义和常识可知，listing_id对预测目标 interest_level 没有用，去掉。

其中 building_id 对预测目标没用，可以直接删除。

photos 应该有用，但是目前的分类模型用不上，也删除。

display_address 、street_address 和 latitude、longitude 都是位置信息，对预测目标比较重要，可以选择两种之一。

latitude、longitude 方便计算，留下。display_address 、street_address 删除。

In [23]:
drop_list = ['building_id', 'photos', 'display_address', 'street_address', 'listing_id']
train.drop(drop_list, axis=1, inplace= True)
train.drop(['interest_level'], axis=1, inplace= True)
test.drop(drop_list, axis=1,inplace= True)

### 下面对特征的处理，参考老师代码

In [24]:
# 去除离群点
def remove_noise(df):
#remove some noise
    df= df[df.price < 10000]

    df.loc[df["bathrooms"] == 112, "bathrooms"] = 1.5
    df.loc[df["bathrooms"] == 10, "bathrooms"] = 1
    df.loc[df["bathrooms"] == 20, "bathrooms"] = 2

#构造新特征
#price_bathrooms：单位bathroom的价格
#price_bedrooms：单位bedroom的价格
def create_price_room(df):
    df['price_bathrooms'] =  (df["price"])/ (df["bathrooms"] +1.0)
    df['price_bedrooms'] =  (df["price"])/ (df["bedrooms"] +1.0)

#room_diff：bathroom房间数 - bedroom房间数
#room_num：bathroom房间数 + bedroom房间数
def create_room_diff_sum(df):
    df["room_diff"] = df["bathrooms"] - df["bedrooms"]
    df["room_num"] = df["bedrooms"] + df["bathrooms"]

# 创建日期反映房子新旧程度，转换为数字
def procdess_created_date(df):
    df['Date'] = pd.to_datetime(df['created'])
#     df['Year'] = df['Date'].dt.year
#     df['Month'] = df['Date'].dt.month
#     df['Day'] = df['Date'].dt.day
#     df['Wday'] = df['Date'].dt.dayofweek
#     df['Yday'] = df['Date'].dt.dayofyear
#     df['hour'] = df['Date'].dt.hour

    df.drop(['Date', 'created'], axis=1,inplace = True)

#简单丢弃，也可以参照feature特征处理方式：
# 其实房屋描述对预测目标比较重要，但是还没学会这种数据处理方法 
def procdess_description(df):
    df.drop(['description'], axis=1,inplace = True)

# 将manager分为几个等级：物业管理？
# top 1%， 2%， 5， 10， 15， 20， 25， 30， 50，
def procdess_manager_id(df):
    df.info()
    managers_count = df['manager_id'].value_counts()

    df['top_10_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 90)] else 0)
    df['top_25_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 75)] else 0)
    df['top_5_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 95)] else 0)
    df['top_50_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 50)] else 0)
    df['top_1_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 99)] else 0)
    df['top_2_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 98)] else 0)
    df['top_15_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 85)] else 0)
    df['top_20_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 80)] else 0)
    df['top_30_manager'] = df['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
        managers_count.values >= np.percentile(managers_count.values, 70)] else 0)
    
    df.drop(['manager_id'], axis=1,inplace = True)

## latitude, longtitude
聚类降维编码(#用训练数据训练，对训练数据和测试数据都做变换)

到中心的距离（论坛上讨论到曼哈顿中心的距离更好）

In [25]:
def procdess_location_train(df):   
    train_location = df.loc[:,[ 'latitude', 'longitude']]
    
     # Clustering
    kmeans_cluster = KMeans(n_clusters=20)
    res = kmeans_cluster.fit(train_location)
    res = kmeans_cluster.predict(train_location)

    df['cenroid'] = res

    # L1 distance
    center = [ train_location['latitude'].mean(), train_location['longitude'].mean()]
    df['distance'] = abs(df['latitude'] - center[0]) + abs(df['longitude'] - center[1])
    
    #原始特征也可以考虑保留，此处简单丢弃
    df.drop(['latitude', 'longitude'], axis=1, inplace=True)
    
    return kmeans_cluster,center

def procdess_location_test(df, kmeans_cluster, center):   
    test_location = df.loc[:,[ 'latitude', 'longitude']]
    
     # Clustering
    res = kmeans_cluster.predict(test_location)

    df['cenroid'] = res

    # L1 distance
    df['distance'] = abs(df['latitude'] - center[0]) + abs(df['longitude'] - center[1])
    df.drop(['latitude', 'longitude'], axis=1, inplace=True)

## features
描述特征文字长度

特征中单词的词频，相当于以数据集features中出现的词语为字典的one-hot编码（虽然是词频，但在这个任务中每个单词通常只出现一次）

In [26]:
def procdess_features_train_test(df_train, df_test):
    n_train_samples = len(df_train.index)
    
    df_train_test = pd.concat((df_train, df_test), axis=0)
    df_train_test['features2'] = df_train_test['features']
    df_train_test['features2'] = df_train_test['features2'].apply(lambda x: ' '.join(x))

    c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1), decode_error='ignore')
    c_vect_sparse = c_vect.fit_transform(df_train_test['features2'])
    c_vect_sparse_cols = c_vect.get_feature_names()

    df_train.drop(['features'], axis=1, inplace=True)
    df_test.drop(['features'], axis=1, inplace=True)
    
    #hstack作为特征处理的最后一部，先将其他所有特征都转换成数值型特征才能处理,稀疏表示
    df_train_sparse = sparse.hstack([df_train, c_vect_sparse[:n_train_samples,:]]).tocsr()
    df_test_sparse = sparse.hstack([df_test, c_vect_sparse[n_train_samples:,:]]).tocsr()
    
    #常规datafrmae
    tmp = pd.DataFrame(c_vect_sparse.toarray()[:n_train_samples,:],columns = c_vect_sparse_cols, index=df_train.index)
    df_train = pd.concat([df_train, tmp], axis=1)
    
    tmp = pd.DataFrame(c_vect_sparse.toarray()[n_train_samples:,:],columns = c_vect_sparse_cols, index=df_test.index)
    df_test = pd.concat([df_test, tmp], axis=1)
    
    #df_test = pd.concat([df_test, tmp[n_train_samples:,:]], axis=1)
  
    return df_train_sparse,df_test_sparse,df_train, df_test

def procdess_features_test(df, c_vect):
    df['features2'] = df['features']
    df['features2'] = df['features2'].apply(lambda x: ' '.join(x))

    c_vect_sparse = c_vect.transform(df['features2'])
    c_vect_sparse_cols = c_vect.get_feature_names()

    df.drop(['features', 'features2'], axis=1, inplace=True)
    
    #hstack作为特征处理的最后一部，先将其他所有特征都转换成数值型特征才能处理
    df_sparse = sparse.hstack([df, c_vect_sparse]).tocsr()
    
    tmp = pd.DataFrame(c_vect_sparse.toarray(),columns = c_vect_sparse_cols, index=df.index)
    df = pd.concat([df, tmp], axis=1)
    
    return df_sparse, df

In [27]:
# 标准化
def transform_train(df):
    # 分离出数字型特征、类别型特征
    numerical_features = df.select_dtypes(exclude = ["object"]).columns
    df_num = df[numerical_features]
    
    cat_features = df.select_dtypes(include = ["object"]).columns
    df_cat = df[cat_features]
    
    # 初始化特征的标准化器
    ss_X = StandardScaler()

    # 分别对训练和测试数据的特征进行标准化处理
    # fit 是对训练数据集计算 mean 和 std
    temp = ss_X.fit_transform(df_num)
    df_num = pd.DataFrame(data=temp, columns=numerical_features, index =df_num.index)
    df_cat = pd.DataFrame(data=df_cat, columns=cat_features, index =df_cat.index)
    
    return df_num, df_cat, ss_X

def transform_test(df, ss_X):
    # 分离出数字型特征、类别型特征
    numerical_features = df.select_dtypes(exclude = ["object"]).columns
    df_num = df[numerical_features]
    
    cat_features = df.select_dtypes(include = ["object"]).columns
    df_cat = df[cat_features]
    
    #用训练好的标准化器，对测试数据进行标准化 
    temp = ss_X.transform(df_num)
    df_num = pd.DataFrame(data=temp, columns=numerical_features, index =df_num.index)
    df_cat = pd.DataFrame(data=df_cat, columns=cat_features, index =df_cat.index)
    
    return df_num, df_cat

## 对训练样本做特征工程
执行上面定义的函数

In [28]:
remove_noise(train)

create_price_room(train)
create_room_diff_sum(train)

procdess_created_date(train)

procdess_description(train)

# 先对上面的数字类型特征进行标准化，再处理下面的类别型特征
train_num, train_cat, ss_X = transform_train(train)

procdess_manager_id(train_cat)

#还是直接用经纬度
#kmeans_cluster,center = procdess_location_train(train)

D:\Applications\Anaconda\envs\python3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 10 to 99994
Data columns (total 2 columns):
features      49352 non-null object
manager_id    49352 non-null object
dtypes: object(2)
memory usage: 1.1+ MB


## 对测试样本做特征工程

In [29]:
remove_noise(test)

create_price_room(test)
create_room_diff_sum(test)

procdess_created_date(test)

procdess_description(test)

# 先对上面的数字类型特征进行标准化，再处理下面的类别型特征
test_num, test_cat = transform_test(test, ss_X)

procdess_manager_id(test_cat)

#还是直接用经纬度
#procdess_location_test(test, kmeans_cluster, center)

X_train_sparse,X_test_sparse,train_cat,test_cat = procdess_features_train_test(train_cat,test_cat)

D:\Applications\Anaconda\envs\python3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74659 entries, 0 to 99999
Data columns (total 2 columns):
features      74659 non-null object
manager_id    74659 non-null object
dtypes: object(2)
memory usage: 1.7+ MB


## 特征处理结果存为文件

In [30]:
#存为csv格式方便用excel查看(属性名字有重复，features得到的词语中也有bathrooms和bedrooms)
train = pd.concat([train_num, train_cat], axis=1)
test= pd.concat([test_num, test_cat],axis = 1)

train = pd.concat([train, y_train], axis=1)
test= pd.concat([test_id, test],axis = 1)
train.to_csv(dpath+'RentListingInquries_FE_train.csv', index=False)
test.to_csv(dpath+'RentListingInquries_FE_test.csv', index=False)